In [ ]:
%matplotlib inline
import warnings

warnings.filterwarnings("ignore")
from IPython.display import display
# tag: remove-cell applied

# Tumor and Multitumor Data

Conducting dose-response analysis on dichotomous tumor data differs from analyzing standard dichotomous tumor data in the following ways:

* The Multistage cancer model uses different parameter settings for model fit than the standard Multistage model.
* A cancer slope factor is calculated.
* In some cases, there may be a need to combine multiple tumor datasets and then calculate a single cancer slope factor.

To that end, this guide covers some different approaches that you can use in `pybmds` for handling tumor data.

## Quickstart

To run a single dataset:

In [ ]:
import pybmds
from pybmds.models.dichotomous import MultistageCancer

dataset = pybmds.DichotomousDataset(
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
    name="Tumor dataset A",
    dose_units="mg/kg-d",
)

model = MultistageCancer(dataset, settings={"bmr": 0.1})
model.execute(slope_factor=True)

print(f"BMD = {model.results.bmd:f}")
print(f"BMDL = {model.results.bmdl:f}")
print(f"CSF = {model.results.slope_factor:f}")

model.plot()

To run multiple datasets and calculate a single combined slope factor:

In [ ]:
import pybmds

datasets = [
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 1, 7, 15, 19],
        name="Tumor A",
        dose_units="mg/m³",
    ),
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 0, 1, 7, 11],
        name="Tumor B",
        dose_units="mg/m³",
    ),
]

session = pybmds.Multitumor(datasets, settings={"bmr": 0.2}, name="Example")
session.execute()

session.plot()

To view individual model results for selected models for each dataset:

In [ ]:
# Print overall results
print("Overall")
print(f"BMD = {session.results.bmd:f}")
print(f"BMDL = {session.results.bmdl:f}")
print(f"CSF = {session.results.slope_factor:f}")
print()

# Print individual model results
selected_model_indexes = session.results.selected_model_indexes
for i, dataset_models in enumerate(session.models):
    selected_index = selected_model_indexes[i]
    selected_model = dataset_models[selected_index]
    print(f"{selected_model.dataset.metadata.name}: {selected_model.name()}")
    print(f"BMD = {selected_model.results.bmd:f}")
    print(f"BMDL = {selected_model.results.bmdl:f}")
    print(f"CSF = {selected_model.results.slope_factor:f}")
    print()

## Create a tumor dataset

Create a tumor dataset using the same method as a dichotomous dataset.

As with a dichotomous dataset, provide a list of doses, incidences, and the total number of subjects, one item per dose-group. 

You can also add optional attributes, such as `name`, `dose_name`, `dose_units`, `response_name`, `response_units`, etc.

In [ ]:
dataset = pybmds.DichotomousDataset(
    name="Chemical X Tumor A",
    dose_units="ppm",
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

print(dataset.tbl())
dataset.plot()

## Single dataset fit

With a single tumor dataset defined above, you can run a single Multistage cancer model:

In [ ]:
import pybmds
from pybmds.models.dichotomous import MultistageCancer

model = MultistageCancer(dataset, settings={"bmr": 0.10, "degree": 2})
model.execute(slope_factor=True)
model.plot()

After executing, results are stored in a `results` attribute on the model. You can view individual items in the results by accessing:

In [ ]:
print(model.name())
print(f"BMD = {session.results.bmd:f}")
print(f"BMDL = {session.results.bmdl:f}")
print(f"CSF = {session.results.slope_factor:f}")

Or generate a text report to view a summary:

In [ ]:
print(model.text())

### Change input settings

Model settings can be customized for a run, as with standard dichotomous models.

In [ ]:
model = MultistageCancer(
    dataset, 
    settings={
        "bmr_type": pybmds.DichotomousRiskType.AddedRisk, 
        "bmr": 0.15, 
        "degree": 3,
    },
)
print(model.settings.tbl())

### Change parameter settings

Initial parameter settings are different for the `MultistageCancer` model compared with the dichotomous `Multistage`:

In [ ]:
from pybmds.models.dichotomous import Multistage, MultistageCancer

model = Multistage(dataset)
print("Multistage parameter settings:")
print(model.priors_tbl())

model = MultistageCancer(dataset)
print("Multistage Cancer parameter settings:")
print(model.priors_tbl())

For Multistage models, the `b2` parameter setting is reused for all beta parameters greater than or equal to b2.

These can be updated:

In [ ]:
model.settings.priors.update("g", initial_value=0, min_value=-10, max_value=10)
model.settings.priors.update("b1", initial_value=10, min_value=0, max_value=100)
model.settings.priors.update("b2", initial_value=20, min_value=0, max_value=1000)

print(model.priors_tbl())

### Fit multiple models

The previous example runs a single Multitumor model to a single dataset. However, you may want, for example, to run multiple multitumor models of varying degrees to a single dataset. 

## Multiple dataset fit

To fit multiple models and one or more datasets, use an instance of the Multitumor class:

In [ ]:
import pybmds

datasets = [
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 1, 7, 15, 19],
        name="Tumor A",
        dose_units="mg/m³",
    ),
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 0, 1, 7, 11],
        name="Tumor B",
        dose_units="mg/m³",
    ),
]

session = pybmds.Multitumor(datasets)
session.execute()

print(session.results.tbl())
session.plot()

You can generate Excel and Word exports:

In [ ]:
# save excel report
df = session.to_df()
df.to_excel("output/report.xlsx")

# save to a word report
report = session.to_docx()
report.save("output/report.docx")

### Change model settings

Settings for all datasets and models should be configured globally and are applied to all models:

In [ ]:
session = pybmds.Multitumor(datasets, settings={
    "bmr_type": pybmds.DichotomousRiskType.AddedRisk, 
    "bmr": 0.15,
})

### Change model degree

By default, multiple models are executed for each dataset, where the degree is varied from 1 to the number of doses minus 1 (and a maximum of 8). 

For this example, we first create three datasets:

In [ ]:
datasets = [
    pybmds.DichotomousDataset(
        doses=[0, 2, 3, 4, 5, 6, 7, 8, 9],
        ns=[20, 20, 20, 20, 20, 20, 20, 20, 20],
        incidences=[0, 1, 4, 8, 11, 12, 13, 14, 15],
        name="Tumor A (9 groups)",
        dose_units="mg/m³",
    ),
    pybmds.DichotomousDataset(
        doses=[0, 2, 3, 4, 5, 6, 7, 8, 9],
        ns=[20, 20, 20, 20, 20, 20, 20, 20, 20],
        incidences=[0, 1, 7, 15, 19, 19, 19, 19, 19],
        name="Tumor B (9 groups)",
        dose_units="mg/m³",
    ),
    pybmds.DichotomousDataset(
        doses=[0, 2, 3, 4, 5],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 0, 1, 7, 11],
        name="Tumor C (5 groups)",
        dose_units="mg/m³",
    ),
]

Next, we specify which model degrees to run for each dataset using `degrees`. Setting a value of 0 runs all degrees available up to a maximum of 8; specifying a specific degree will only run the specified degree.

In [ ]:
degrees = [0, 3, 2]
session = pybmds.Multitumor(datasets, degrees=degrees)
session.execute()
session.plot()

The analysis executed the following models for each dataset:

In [ ]:
for dataset_models in session.models:
    print(f"{dataset_models[0].dataset.metadata.name}")
    for model in dataset_models:
        print("\t" + model.name())